In [1]:
import tensorflow
print(tensorflow.__version__)

1.15.2-dlenv_tfe


### CNN for Image Recognition with MNIST Dataset

In [2]:
# Import Python Libraries: NumPy and Pandas
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import tensorflow as tf

In [4]:
from tensorflow.examples.tutorials.mnist import input_data

In [5]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [6]:
type(mnist)

tensorflow.contrib.learn.python.learn.datasets.base.Datasets

In [7]:
mnist.train.num_examples

55000

In [8]:
mnist.test.num_examples

10000

### Preparation for Building CNN Model: Define Supporting Functions

### Initialize Weights in filter

In [9]:
# This function returns a tf variable used to store weights in a filter
# This variable is initialized with values that can be used to initialize weights
# The values are random numbers

def initialize_weights(filter_shape):
    init_random_dist = tf.truncated_normal(filter_shape, stddev=0.1)
    return(tf.Variable(init_random_dist))

### Initialize bias

In [10]:
# This function returns a tf variable used to store weights in a bias
# This variable is initialized with values that can be used to initialize bias
# The values is initialized to 0.1

def initialize_bias(bias_shape):
    initial_bias_vals = tf.constant(0.1, shape=bias_shape)
    return(tf.Variable(initial_bias_vals))

### Set Up a Convolutional Layer and perform convolution computation: Dot Product(x*W)

In [11]:
# Define a function to set up a convolution layer (conv2d)
# Parameters:
# -inputs: [batch, H, W, Channels]
# -filter_shape:
# [filter H, filter w, in_channels (in_depth = in_num_filter), out_channels (out_depth = out_num_filters)]
# for example: [5, 5, 1, 32]
# Return: Outputs of the layer: The dot product: inputs * weight: x * W

def create_convolution_layer_and_compute_dot_product(inputs, filter_shape):
    
    # initialize the weights in the filter
    filter_initialized_with_weights = initialize_weights(filter_shape)
    
    # Create a convolution layer
    conv_layer_outputs = tf.nn.conv2d(inputs, filter_initialized_with_weights, strides=[1, 1, 1, 1], padding='SAME')
    
    # Return the convolution layer outputs
    return (conv_layer_outputs)    
    

### Set Up a ReLU Layer and perform convolution computation: Dot Product + Bias(x.W+b)

In [12]:
# Define a function
# First, to set up a ReLU layer: an activation function layer
# then, perform the computation: dot product + bias: (x*W)+b

# ReLU: Rectified Linear Unit - a popular activation function used in CNN
# In this layer, the bias has been initialized
# Paramters: 
# inputs - outputs from the preceding convolution layer: Dot product inputs * weights
# filter_shape:
# [filter H, filter W, in_channels(in_depth = in_num_filters), out_channels(out_depth = out_num_filters)]
# for example; [5, 5, 1, 32]

# Return: Outputs of the layer: dot product + b: (x * W) + b

def create_relu_and_compute_dotproduct_plus_b(inputs, filter_shape):
    # Initialize bias for each input channel
    b = initialize_bias([filter_shape[3]])
    
    # Perform the computation first by adding: inputs (x*W)+b
    # Create a ReLU layer associated with the preceding convolution layer
    relu_layer_outputs = tf.nn.relu(inputs + b)
    
    # Return the putputs of the ReLU layer
    return(relu_layer_outputs)

### Set Up a pooling layer and reduce spatial size

In [13]:
# Define a function
# First, to set up a pooling layer
# then, to reduce the spatial size of the inputs data

# Pooling method: Max pooling
# Kernal size: 2x2
# stride : 2

# Parameters:
# inputs: outputs of teh preceding layer
# return: outputs of the layer

def create_maxpool2by2_and_reduce_spatial_size(inputs):
    # create a pooling layer
    pooling_layer_outputs = tf.nn.max_pool(inputs, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    # Return the pooling layer
    return(pooling_layer_outputs)


### Set Up a Fully connected Layer and perform computation: (inputs * weights) + bias

In [14]:
# Define a function to set up a fully connected layer
# Also perform computation: (Dot product (inputs*weights))+bias
# Return the results
# Parameters:
# inputs: outputs of the preceding layer
# size: the size of the outputs, i.e; number of out_channels

def create_fully_connected_layer_and_compute_dotproduct_plus_bias(inputs, output_size):
    # Get the number of input channels from the input
    # Inputs: the outputs from the preceding layer or previous operation like reshaping
    input_size = int(inputs.get_shape()[1])
    
    # initialize the weights of the filter of the FC layer
    # Filter shape [in_channels, out_channels]
    # Each weight for one filter cell
    W = initialize_weights([input_size, output_size])
    
    # Initialize_weights([input_size, output_size])
    b = initialize_bias ([output_size])
    
    # First: perform the computation for the FC layer: Dot product inputs * W
    # then: add bias to get the results: outputs of the FC layer
    fc_xW_plus_bias_outputs = tf.matmul(inputs, W)+b
    
    # return the results: outputs
    return(fc_xW_plus_bias_outputs)


### Phase I: Build the Convolutional Neural Network

### Create Placeholders for Inputs and Labels: x & y_true

In [15]:
# PLaceholder

# Create a placeholder for the pnputs data: x
# x: a 2D array 
# x: a placeholder that can hold any number of rows/record
# Each row is a vector (1D array) to hold data for one image
# Each row/record/image has 784 values/elements: 1 pixel = 1 value

x = tf.placeholder(tf.float32, shape=[None, 784])

In [16]:
# PLaceholder
# Create a placeholder for the labels of the inputs data: y_true
# y_true: a 2D array
#y_true: can hold any number of row/records
# Each row is the label of one image, i.e; one digit
# Each row/record: The label is stored in the one-hot format
# One hot format: [1 0 0 0 0 0 0 0 0 0] -> 0; [0 0 0 0 0 0 0 1 0] -> 8

y_true = tf.placeholder(tf.float32, [None, 10])

### Reshape the input placeholder x

In [17]:
# Prepare feeding inputs into the 1st conv layer
# Reshape the pnput x: A placeholder
# From 1D array (vector) -> Original input shape: 4D-input: [batch, H, W, depth channels]
# Depth = color channels: grey scale = 1
# Reshaped inputs: x_image: [ 1, 28, 28, 1]

x_image = tf.reshape(x, [-1, 28, 28, 1])

### Create 1st Convolutional Layer, ReLU Layer, and Perform Computation: x*W + b

### Reshape the input placeholder x

In [18]:
# Create the 1st convilution layer
# Inputs: x_image: Reshaped inputs with shape [1, 28, 28, 1]
# filter_shape: [5, 5, 1, 32]
# filter: 5x5
# input channels: 1
# output channels: 32

# Create the 1st convolution layer
# then lean/extract the features, get teh results (outputs): Dot product of inputs * weights
# Return the outputs of the layer
conv_layer_1_outputs = create_convolution_layer_and_compute_dot_product(x_image, filter_shape=[5, 5, 1, 32])

# Create the ReLU layer for the first convolution layer
# Accept the outputs from the 1st conv layer as the inputs
# Perform the computation at the layer: add inputs + bias
# Return teh outputs of the layer
conv_relu_layer_1_outputs = create_relu_and_compute_dotproduct_plus_b(conv_layer_1_outputs, filter_shape=[5, 5, 1, 32])

### Create 1st pooling layer and reduce spatial size

In [19]:
# Create the 1st pooling layer
# then reduce the spatial size of the input data
# Return: output of the layer

pooling_layer_1_outputs = create_maxpool2by2_and_reduce_spatial_size(conv_relu_layer_1_outputs)

### Create 2nd Convolutional Layer, ReLU Layer, and Perform Computation: x*W + b

In [20]:
# Create the 1st convilution layer
# Inputs: x_image: Reshaped inputs with shape [1, 28, 28, 1]
# filter_shape: [5, 5, 1, 32]
# filter: 5x5
# input channels: 1
# output channels: 32

# Create the 2nd convolution layer
# then lean/extract the features, get teh results (outputs): Dot product of inputs * weights
# Return the outputs of the layer
conv_layer_2_outputs = create_convolution_layer_and_compute_dot_product(pooling_layer_1_outputs, filter_shape=[5, 5, 32, 64])

# Create the ReLU layer for the first convolution layer
# Accept the outputs from the 1st conv layer as the inputs
# Perform the computation at the layer: add inputs + bias
# Return teh outputs of the layer
conv_relu_layer_2_outputs = create_relu_and_compute_dotproduct_plus_b(conv_layer_2_outputs, filter_shape=[5, 5, 32, 64])

### Create 2nd pooling layer and reduce spatial size

In [21]:
# Create the 1st pooling layer
# then reduce the spatial size of the input data
# Return: output of the layer

pooling_layer_2_outputs = create_maxpool2by2_and_reduce_spatial_size(conv_relu_layer_2_outputs)

### Reshape/Flatten Data Making it ready to be fed into 1st FC layer

In [22]:
# Reshape and flatten the output of the 2nd pooling layer
# Prepare to feed the output data into the 1st fully connected layer

pooling_layer_2_outputs_flat = tf.reshape(pooling_layer_2_outputs, [-1, 7*7*64])

### Create 1st FC Layer, ReLU Layer, and output data to dropout layer

In [23]:
# First create the FC (Fully Connected) Layer
# Feed the output (already flattened) of the 2nd pooling layer as the inputs into this layer
# then perform the computation: dot product + bias: (x * W) + b

# Parameters:
# pooling_layer_2_outputs_flat
# output_size: 1024 out channels

# Return: outputs of the computation: (x * W) + b

fc_layer_1_outputs = create_fully_connected_layer_and_compute_dotproduct_plus_bias (pooling_layer_2_outputs_flat, output_size=1024)

# Create the ReLU layer of the 1st FC layer
# VIP Notes: The activation function of this layer is also ReLU

# Return: Outputs of the layer
fc_relu_layer_1_outputs = tf.nn.relu(fc_layer_1_outputs)

### Create Dropout Layer and Dropout a Fraction of outputs randomly

In [24]:
# Declare a placeholder to hold the alue of probability (percentage %) to keep:
# i.e; the percentage % of total output channels that will be kept
# e.g.: keep_prob = hold_prob = 50% --> Keep 50% --> Dropout 50%
# Which nodes/channels to keep or dropout: Selected randomly

hold_prob = tf.placeholder(tf.float32)

# Dropout:
# Set the outputs to 0 (zero) so that will be ignored in the next layer

fc_dropout_outputs = tf.nn.dropout(fc_relu_layer_1_outputs, keep_prob=hold_prob)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


### Create Fianl FC Layer, Compute (x.W + b), and produce final outputs

In [25]:
# Create the final FC layer
# then compute: x*W + b
# Parameters:
# fc_dropout_outputs: Outputs from the dropout layer

# Return y_pred: final predicted outputs, i.e., final classification outputs
y_pred = create_fully_connected_layer_and_compute_dotproduct_plus_bias(fc_dropout_outputs, output_size=10)

### Define loss function and calculate softmax cross entropy loss

In [26]:
# Define loss function: cross-entropy with logits, i.e., with the final outputs
# Calculate the softmax cross-entropy loss

softmax_cross_entropy_loss = tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)

# Compute the mean of losses
cross_entropy_mean = tf.reduce_mean(softmax_cross_entropy_loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



### Create an optimizer to optimize CNN Model and set learning rate

In [27]:
# Get an ADAM optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)


### Create a Trainer to Training CNN Model

In [28]:
# Create a CNN model trainer that can train the model
# And optimize the model by minimizing the softmax cross_entropy loss

cnn_trainer = optimizer.minimize(cross_entropy_mean)

### Train and Test CNN Deep Learning Model on MNIST Dataset

### Create a variable initializer to Initialize all variables

In [29]:
# Get a variable initializer

vars_initializer = tf.global_variables_initializer()

### Set the steps

In [30]:
# x: mnist.train: 50000 images
# Each time of training (run the whole process) = 1 step
# Each time of training: use one batch of inputs

# Steps = 5000: Fininsh the training after running 5000 times

steps = 5000

### Run tf.session() to Train and Test Deep Learning CNN Model

In [31]:
with tf.Session() as sess:
    # First, run vars_initializer to initialize
    sess.run(vars_initializer)
    
    for i in range(steps):
        # Each batch: 50 images
        batch_x, batch_y = mnist.train.next_batch(50)
        
        # Train the model
        # Dropout keep_prob (% to keep): 0.5 --> 50% will be dropped out
        sess.run(cnn_trainer, feed_dict={x: batch_x, y_true: batch_y, hold_prob: 0.5})
        
        # Test the model: at each 100th step
        # Run this block of code for each 100 times of training, each time run a batch 
        if i % 100 == 0:
            print('ON STEP: {}'.format(i))
            print('ACCURACY: ')
            
            # Compare to find matches of y_pred and y_true
            matches = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
            
            # Cast the matches from integers to tf.float32
            # Calculate the accuracy using the mean of matches
            acc = tf.reduce_mean(tf.cast(matches, tf.float32))
            
            # Test the model at each 100th step
            # Using test dataset
            # Dropout: NONE because of test, not training
            test_accuracy = sess.run (acc, feed_dict = {x: mnist.test.images, \
                                                        y_true: mnist.test.labels, \
                                                        hold_prob: 1.0})
            print(test_accuracy)
            print('\n')

ON STEP: 0
ACCURACY: 
0.1737


ON STEP: 100
ACCURACY: 
0.9475


ON STEP: 200
ACCURACY: 
0.9648


ON STEP: 300
ACCURACY: 
0.9721


ON STEP: 400
ACCURACY: 
0.9771


ON STEP: 500
ACCURACY: 
0.9767


ON STEP: 600
ACCURACY: 
0.9788


ON STEP: 700
ACCURACY: 
0.9791


ON STEP: 800
ACCURACY: 
0.9799


ON STEP: 900
ACCURACY: 
0.9765


ON STEP: 1000
ACCURACY: 
0.9844


ON STEP: 1100
ACCURACY: 
0.9838


ON STEP: 1200
ACCURACY: 
0.986


ON STEP: 1300
ACCURACY: 
0.9885


ON STEP: 1400
ACCURACY: 
0.9857


ON STEP: 1500
ACCURACY: 
0.987


ON STEP: 1600
ACCURACY: 
0.9859


ON STEP: 1700
ACCURACY: 
0.9889


ON STEP: 1800
ACCURACY: 
0.9885


ON STEP: 1900
ACCURACY: 
0.9848


ON STEP: 2000
ACCURACY: 
0.9865


ON STEP: 2100
ACCURACY: 
0.987


ON STEP: 2200
ACCURACY: 
0.9878


ON STEP: 2300
ACCURACY: 
0.987


ON STEP: 2400
ACCURACY: 
0.9898


ON STEP: 2500
ACCURACY: 
0.9885


ON STEP: 2600
ACCURACY: 
0.9892


ON STEP: 2700
ACCURACY: 
0.9882


ON STEP: 2800
ACCURACY: 
0.9897


ON STEP: 2900
ACCURACY: 
0.990